In [3]:
# Spark setup - alternative setup from Slack as original was not working.
from pyspark.sql import SparkSession
spark_session = SparkSession.builder \
    .master("spark://192.168.2.156:7077") \
    .appName("projectgroup1_reddit-dataset-100k") \
    .config("spark.dynamicAllocation.enabled", True) \
    .config("spark.dynamicAllocation.shuffleTracking.enabled", True) \
    .config("spark.shuffle.service.enabled", False) \
    .config("spark.dynamicAllocation.executorIdleTimeout", "30s") \
    .config("spark.executor.cores", 2) \
    .config("spark.driver.port",9999)\
    .config("spark.blockManager.port",10005)\
    .config("spark.jars.packages", "com.johnsnowlabs.nlp:spark-nlp_2.12:5.5.3")\
    .getOrCreate()

25/03/06 07:20:42 WARN SparkSession: Using an existing Spark session; only runtime SQL configurations will take effect.


In [5]:
# Load Reddit dataset which is up on Spark.
file_path = "hdfs://192.168.2.156:9000/data/reddit/reddit_100k.json" # For now use a smaller subset to simply test.

df = spark_session.read.json(file_path)
rows_1 = df.count()
print(f"Current amount of rows: {rows_1}")
df.show(2)

Current amount of rows: 200001
+---------------+----------------+--------------------+--------------------+-----------+-------+--------------------+---------+------------+--------------------+-----------+-----+
|_corrupt_record|          author|                body|             content|content_len|     id|      normalizedBody|subreddit|subreddit_id|             summary|summary_len|title|
+---------------+----------------+--------------------+--------------------+-----------+-------+--------------------+---------+------------+--------------------+-----------+-----+
|           NULL|raysofdarkmatter|I think it should...|I think it should...|        178|c69al3r|I think it should...|     math|    t5_2qh0n|Shifting seasonal...|          8| NULL|
|               |            NULL|                NULL|                NULL|       NULL|   NULL|                NULL|     NULL|        NULL|                NULL|       NULL| NULL|
+---------------+----------------+--------------------+--------------

In [6]:
from pyspark.sql.functions import col
# Select the only two we wanna analyze, the subreddit name and it's contents.
df = df.select(col("body"), col("subreddit"))
rows_2 = df.count()
print(f"Current amount of rows: {rows_2}")
df.show(5)

Current amount of rows: 200001
+--------------------+-----------+
|                body|  subreddit|
+--------------------+-----------+
|I think it should...|       math|
|                NULL|       NULL|
|Art is about the ...|      funny|
|                NULL|       NULL|
|Ask me what I thi...|Borderlands|
+--------------------+-----------+
only showing top 5 rows



In [7]:
# Cleanup the data by removing null rows as we can see some above.
df = df.dropna(subset=["body", "subreddit"])
rows_3 = df.count()
rows_dropped = rows_2 - rows_3
print(f"Current amount of rows: {rows_3}, NULL rows dropped: {rows_dropped}.")
df.show(10)

Current amount of rows: 99668, NULL rows dropped: 100333.
+--------------------+--------------------+
|                body|           subreddit|
+--------------------+--------------------+
|I think it should...|                math|
|Art is about the ...|               funny|
|Ask me what I thi...|         Borderlands|
|In Mechwarrior On...|            gamingpc|
|You are talking a...|              Diablo|
|All but one of my...|   RedditLaqueristas|
|I could give a sh...|               apple|
|So you're saying ...|               apple|
|I love this idea ...|RedditFilmsProduc...|
|Theres an entire ...|       AbandonedPorn|
+--------------------+--------------------+
only showing top 10 rows



In [8]:
# Preprocess the data - in accordance with SparkNLP tutorial
from pyspark.sql.functions import regexp_replace, length # Function to replace substrings in column using regular expressions and just check length.

# Remove empty strings (i.e containing only whitespaces)
df = df.filter(~col("body").rlike("^\s*$"))
# Remove Non-ASCII characters like emojis and non-english characters and replaces with spaces.
df = df.withColumn("body", regexp_replace(col("body"), "[^\x00-\x7F]+", " ")) 
# Keep comments with at least 5 letters in one word.
df = df.filter(col("body").rlike("[a-zA-Z]{5,}"))  
# Remove punctuation and replace with an empty string to keep words intact
df = df.withColumn("body", regexp_replace(col("body"), "[!\"#$%&'()*+,-./:;<=>?@\\[\\\\\\]^_`{|}~]", ""))
# Removes very short comments that don't give any meaning really.
df = df.filter(length(col("body")) > 10)
rows_4 = df.count()
print(f"Rows after pre-processing finalized: {rows_4}, total rows dropped during pre-processing: {rows_2 - rows_4}")
df.show(10)
# After this pre-processing has been finalized

[Stage 14:======================================>                   (4 + 2) / 6]

Rows after pre-processing finalized: 99556, total rows dropped during pre-processing: 100445


+--------------------+--------------------+
|                body|           subreddit|
+--------------------+--------------------+
|I think it should...|                math|
|Art is about the ...|               funny|
|Ask me what I thi...|         Borderlands|
|In Mechwarrior On...|            gamingpc|
|You are talking a...|              Diablo|
|All but one of my...|   RedditLaqueristas|
|I could give a sh...|               apple|
|So youre saying t...|               apple|
|I love this idea ...|RedditFilmsProduc...|
|Theres an entire ...|       AbandonedPorn|
+--------------------+--------------------+
only showing top 10 rows



In [2]:
import sparknlp
from sparknlp.base import *
from sparknlp.annotator import *
from sparknlp.pretrained import PretrainedPipeline


from pyspark.ml import Pipeline
from pyspark.sql import SparkSession
import pyspark.sql.functions as F
 
spark = sparknlp.start()

document = DocumentAssembler() \
    .setInputCol("text") \
    .setOutputCol("document")
token = Tokenizer() \
    .setInputCols(["document"]) \
    .setOutputCol("token")
normalizer = Normalizer() \
    .setInputCols(["token"]) \
    .setOutputCol("normal")
vivekn = ViveknSentimentModel.pretrained() \
    .setInputCols(["document", "normal"]) \
    .setOutputCol("result_sentiment")
finisher = Finisher() \
    .setInputCols(["result_sentiment"]) \
    .setOutputCols("final_sentiment")

25/03/06 07:18:21 WARN SparkSession: Using an existing Spark session; only runtime SQL configurations will take effect.


sentiment_vivekn download started this may take some time.
Approximate size to download 873.6 KB
[ | ]

25/03/06 07:18:40 WARN S3AbortableInputStream: Not all bytes were read from the S3ObjectInputStream, aborting HTTP connection. This is likely an error and may result in sub-optimal behavior. Request only the bytes you need via a ranged GET or drain the input stream after use.
25/03/06 07:18:40 WARN S3AbortableInputStream: Not all bytes were read from the S3ObjectInputStream, aborting HTTP connection. This is likely an error and may result in sub-optimal behavior. Request only the bytes you need via a ranged GET or drain the input stream after use.


sentiment_vivekn download started this may take some time.
Approximate size to download 873.6 KB
Download done! Loading the resource.
[ / ]

[Stage 0:>                                                          (0 + 0) / 1]

...

[Stage 0:>                                                          (0 + 1) / 1]

[ \ ]

25/03/06 07:19:07 WARN TaskSetManager: Lost task 0.0 in stage 0.0 (TID 0) (192.168.2.60 executor 0): java.io.FileNotFoundException: File file:/home/ubuntu/cache_pretrained/sentiment_vivekn_en_2.0.2_2.4_1556663184035/metadata/part-00000 does not exist
	at org.apache.hadoop.fs.RawLocalFileSystem.deprecatedGetFileStatus(RawLocalFileSystem.java:779)
	at org.apache.hadoop.fs.RawLocalFileSystem.getFileLinkStatusInternal(RawLocalFileSystem.java:1100)
	at org.apache.hadoop.fs.RawLocalFileSystem.getFileStatus(RawLocalFileSystem.java:769)
	at org.apache.hadoop.fs.FilterFileSystem.getFileStatus(FilterFileSystem.java:462)
	at org.apache.hadoop.fs.ChecksumFileSystem$ChecksumFSInputChecker.<init>(ChecksumFileSystem.java:160)
	at org.apache.hadoop.fs.ChecksumFileSystem.open(ChecksumFileSystem.java:372)
	at org.apache.hadoop.fs.ChecksumFileSystem.lambda$openFileWithOptions$0(ChecksumFileSystem.java:896)
	at org.apache.hadoop.util.LambdaUtils.eval(LambdaUtils.java:52)
	at org.apache.hadoop.fs.ChecksumF


An error occurred while calling z:com.johnsnowlabs.nlp.pretrained.PythonResourceDownloader.downloadModel.
: org.apache.spark.SparkException: Job aborted due to stage failure: Task 0 in stage 0.0 failed 4 times, most recent failure: Lost task 0.3 in stage 0.0 (TID 3) (192.168.2.60 executor 0): java.io.FileNotFoundException: File file:/home/ubuntu/cache_pretrained/sentiment_vivekn_en_2.0.2_2.4_1556663184035/metadata/part-00000 does not exist
	at org.apache.hadoop.fs.RawLocalFileSystem.deprecatedGetFileStatus(RawLocalFileSystem.java:779)
	at org.apache.hadoop.fs.RawLocalFileSystem.getFileLinkStatusInternal(RawLocalFileSystem.java:1100)
	at org.apache.hadoop.fs.RawLocalFileSystem.getFileStatus(RawLocalFileSystem.java:769)
	at org.apache.hadoop.fs.FilterFileSystem.getFileStatus(FilterFileSystem.java:462)
	at org.apache.hadoop.fs.ChecksumFileSystem$ChecksumFSInputChecker.<init>(ChecksumFileSystem.java:160)
	at org.apache.hadoop.fs.ChecksumFileSystem.open(ChecksumFileSystem.java:372)
	at org

Py4JJavaError: An error occurred while calling z:com.johnsnowlabs.nlp.pretrained.PythonResourceDownloader.downloadModel.
: org.apache.spark.SparkException: Job aborted due to stage failure: Task 0 in stage 0.0 failed 4 times, most recent failure: Lost task 0.3 in stage 0.0 (TID 3) (192.168.2.60 executor 0): java.io.FileNotFoundException: File file:/home/ubuntu/cache_pretrained/sentiment_vivekn_en_2.0.2_2.4_1556663184035/metadata/part-00000 does not exist
	at org.apache.hadoop.fs.RawLocalFileSystem.deprecatedGetFileStatus(RawLocalFileSystem.java:779)
	at org.apache.hadoop.fs.RawLocalFileSystem.getFileLinkStatusInternal(RawLocalFileSystem.java:1100)
	at org.apache.hadoop.fs.RawLocalFileSystem.getFileStatus(RawLocalFileSystem.java:769)
	at org.apache.hadoop.fs.FilterFileSystem.getFileStatus(FilterFileSystem.java:462)
	at org.apache.hadoop.fs.ChecksumFileSystem$ChecksumFSInputChecker.<init>(ChecksumFileSystem.java:160)
	at org.apache.hadoop.fs.ChecksumFileSystem.open(ChecksumFileSystem.jav

In [8]:
df_result = \
pipeline.transform(df.withColumnRenamed("body", "text"))

NameError: name 'pipeline' is not defined

In [6]:
spark_session.stop() # Ending our current spark session.